# Spin-1 Non-Abelian Geometric Phase via Floquet Engineering

In [2]:
import numpy as np
import qutip as qt
import matplotlib.pyplot as plt

$$ \hat{\mathcal{H}} = \frac{\tilde{\Omega}}{2}\left( \sin\phi \hat{F}_x + \cos\phi \hat{F}_y \right) + \tilde{\delta} \hat{F}_z $$
Recall the minus plus sign in front of $\tilde{\delta}$ here, should probably be a minus.

$$ \tilde{\Omega} = \Omega_0 \sin\Omega t \cos\omega t $$
$$ \tilde{\delta} = \frac{\Omega_0}{2} \cos\Omega t \cos\omega t $$

$$ \hat{\mathcal{H}}\left( t \right) = \Omega_0 \vec{r} \cdot \hat{\vec{\sigma}} \cos\omega t$$

$$ \vec{r} = \left( \sin\Omega t \cos \Phi, \sin\Omega t \sin \Phi, \cos\Omega t \right)^T $$

In [ ]:
#----- Input Parameters -----
Omega0 = 2*np.pi * 1 #Rabi frequency
Phi = 0 #Operator phase
n_cyc = 2 #Number of Rabi oscillations per op.

#--- Computed Values ---
delta = Omega0/2 #Field detuning
Omega = (1/n_cyc)*Omega0 #Slow freq.
omega = 2*Omega0 #Floquet freq.
fmod = 2*Omega0 #Modulation frequency

#--- Setup Evolution ---
periods = 4 #Number of periods of Omega to simulate over
t = np.linspace(0, periods*(2*np.pi/Omega), num=500) #Time axis

#--- Initial States ---
psi1, psi2, psi3 = qt.basis(3,0), qt.basis(3,1), qt.basis(3,2) #Three-level basis states
psi = psi1 #Initial state
psi = psi.unit() #Force normalization

In [3]:
#----- Global Settings -----
sx, sy, sz = qt.operators.jmat(1, 'x'), qt.operators.jmat(1, 'y'), qt.operators.jmat(1, 'z') #Shorthand spin matrices

#--- Projection Operators ---
p1, p2, p3 = psi1.proj(), psi2.proj(), psi3.proj() #Project onto bare spins (z-basis)
eigx, eigy = sx.eigenstates(), sy.eigenstates() #eigenstates of sx, sy
px1, px2, px3 = eigx[1][0].proj(), eigx[1][1].proj(), eigx[1][2].proj() #Corresponding proj. ops.
py1, py2, py3 = eigy[1][0].proj(), eigy[1][1].proj(), eigy[1][2].proj() #Corresponding proj. ops.

proj_ops = [p1, p2, p3, px1, px2, px3, py1, py2, py3] #All ops. in list